In [ ]:
How many different ways do we find to note the ISO standard and version?

So for any XML reseponse, check for the version.

####Notes

This does not rely on any formal identification of ISO first so any XML that contains any one of the xpath structures is included. This catches ISO records embedded in something like a CSW GetRecords response. And it is only extracting the first value so, if it *is* from a CSW and each ISO within has its own version definition, we aren't capturing that (that would be odd, but wouldn't rule it out).

In [2]:
%reload_ext autoreload
%autoreload 2

import os
import glob
import json
from lxml import etree
from semproc.rawresponse import RawResponse
from semproc.xml_utils import extract_item

def convert_header_list(headers):
    return dict(
        (k.strip().lower(), v.strip()) for k, v in (
            h.split(':', 1) for h in headers)
    )

In [3]:
files = glob.glob('/Users/sparky/Documents/solr_responses/solr_20150922_docs/*.json')

In [4]:
xpaths = [
    (['//*', 'metadataStandardName', 'CharacterString'], ['//*', 'metadataStandardVersion', 'CharacterString']),
    (['//*', 'seriesMetadata', 'MD_Metadata', 'metadataStandardName', 'CharacterString'], ['//*', 'seriesMetadata', 'MD_Metadata', 'metadataStandardVersion', 'CharacterString']),
    (['//*', 'seriesMetadata', 'MI_Metadata', 'metadataStandardName', 'CharacterString'], ['//*', 'seriesMetadata', 'MI_Metadata', 'metadataStandardVersion', 'CharacterString'])
]

In [5]:
versions = set()
responses_containing_version = set()
for f in files:
    with open(f, 'r') as g:
        data = json.loads(g.read())
    
    response = data.get('raw_content')
    headers = convert_header_list(data.get('response_headers', []))
    content_type = headers.get('content-type', '')
    
    rr = RawResponse(response, content_type)
    try:
        content = rr.clean_raw_content()
    except:
        print 'FAILED ', f
        continue

    if rr.datatype != 'xml':
        continue
    
    try:
        xml = etree.fromstring(content)
    except:
        continue
        
    for std_name, std_vsn in xpaths:
        sn = extract_item(xml, std_name)
        sv = extract_item(xml, std_vsn)
        
        if sn or sv:
            versions.add('Standard: {0}, Version: {1}'.format(sn, sv))
            responses_containing_version.add('File: {0}, Standard: {1}, Version: {2}'.format(f.split('/')[-1], sn, sv))
    

FAILED  /Users/sparky/Documents/solr_responses/solr_20150922_docs/14a548cd708b8b3c1b74c47a57dd033bfaf5a7ed.json
FAILED  /Users/sparky/Documents/solr_responses/solr_20150922_docs/2b727b1afb9381062a3f5dd05802e56ce21eeee8.json
FAILED  /Users/sparky/Documents/solr_responses/solr_20150922_docs/3354a2bd8c334dddcb82c4cf50bc18d8619db25e.json
FAILED  /Users/sparky/Documents/solr_responses/solr_20150922_docs/3459af68928a619ac5cc332e251ff72a62bd10d9.json
FAILED  /Users/sparky/Documents/solr_responses/solr_20150922_docs/51bfa74c90c5cacbc46b20cd87abf86aa3a958a3.json
FAILED  /Users/sparky/Documents/solr_responses/solr_20150922_docs/58cc1b77ccf6848cf6f66d3ad75e32efd2df9ad9.json
FAILED  /Users/sparky/Documents/solr_responses/solr_20150922_docs/6269c3680e2d851de9bcd4e9f140783925b77890.json
FAILED  /Users/sparky/Documents/solr_responses/solr_20150922_docs/64aa05e70dda49093c464c1973599aa25d20ecac.json
FAILED  /Users/sparky/Documents/solr_responses/solr_20150922_docs/76d89ea59e67c23c8b16dcc65079410937eb52

In [7]:

versions


{'Standard: ANZLIC Metadata Profile: An Australian/New Zealand Profile of AS/NZS ISO 19115:2005, Geographic information - Metadata, Version: 1.1',
 'Standard: DM - Regole tecniche RNDT, Version: 10 novembre 2011',
 'Standard: GEMINI, Version: 2.1',
 'Standard: Gemini, Version: 2.1',
 'Standard: INSPIRE Implementing Rules for Metadata, Version: 1.2',
 'Standard: ISO 19115 (UK GEMINI), Version: 1.0 (2.2)',
 'Standard: ISO 19115 Geographic Information - Metadata, Version: 2009-02-15',
 'Standard: ISO 19115 Geographic Information - Metadata, Version: ISO 19115',
 'Standard: ISO 19115 Geographic information - Metadata - Converted from Data.gov legacy DMS format., Version: ISO 19115:2003(E)',
 'Standard: ISO 19115 Geographic information - Metadata, Version: ISO 19115:2003(E)',
 'Standard: ISO 19115 Geographic information Metadata; WISE Metadata profile, Version: 2003/Cor.1:2006',
 'Standard: ISO 19115, Version: 2003',
 'Standard: ISO 19115, Version: ISO 19139 / DKRZ ISO Simple Profile V1.0',

In [6]:
with open('outputs/iso_versions_by_response.txt', 'w') as f:
    f.write('\n'.join(responses_containing_version))

In [8]:
len(responses_containing_version)

25371

In [9]:
for v in versions:
    print v

Standard: WMO Core Metadata Profile, Version: 1.0 (draft)
Standard: MEDIN Discovery Metadata Standard, Version: Version 2.3.5
Standard: ISO 19115 Geographic information - Metadata - Converted from Data.gov legacy DMS format., Version: ISO 19115:2003(E)
Standard: MEDIN Discovery Metadata Standard, Version: Version 2.3.7
Standard: ISO 19115/2003/Cor.1:2006, Version: GDI-Vlaanderen Best Practices - versie 1.0
Standard: ISO 19115-2 Geographic Information - Metadata - Part 2: Extensions for Imagery and Gridded Data, Version: ISO 19115-2:2013(E)
Standard: ISO 19115-2 Geographic Information - Metadata - Part 2: Extensions for Imagery and Gridded Data with Biological Extensions, Version: ISO 19115-2:2009(E)
Standard: ISO 19115 Geographic information - Metadata, Version: ISO 19115:2003(E)
Standard: ISO 19115-2 Geographic Information - Metadata Part 2 Extensions for imagery and gridded data, Version: ISO 19115-2:2009(E)
Standard: Gemini, Version: 2.1
Standard: North American Profile of ISO 19115